# TF-MInDi on Mouse Cortex

To illustrate how TF-MInDi works we calculated contribution scores for the top 2000 most specific regions for every cell type of the mouse cortex using 
`CREsted`'s DeepBICCN2 model (see [the model](https://crested.readthedocs.io/en/latest/models/BICCN/deepbiccn2.html) and [specific contribution scores](https://crested.readthedocs.io/en/latest/api/tools/_autosummary/crested.tl.contribution_scores_specific.html) calculation).

That function should give you a directory of contribution scores per cell type and one hot encoded regions per cell type (.npz).  
We used CREsted, but you can use any model and input x output interpretation score calculation methods (such as tangermeme), as long as they are converted to the expected numpy array shape.  

## Fetching motif collections and annotations

On top of the contribution scores and sequences, we'll need motif collections to calculate similarity scores with the extracted seqlets.  
TF-MInDi provides functionality to fetch and load some of the default SCENIC+ motif databases.  
We can also fetch some motif annotations and motif-to-dbd mappings to visualize the seqlets later. 

In [1]:
import tfmindi as tm

import os
import re
import numpy as np
from tqdm import tqdm

In [ ]:
# fetch collection and annotations
motif_collection_dir = tmotif_collection_dir = tm.fetch_motif_collection()
motif_annotations_file = tm.fetch_motif_annotations()

# to speed up all analyses, we can choose to only load a sampled subset of motifs. We have selected some and stored those names in a .txt file.
DATA_DIR = "../../../../data/tfmindi/"
motif_samples_path = DATA_DIR + "sampled_motifs.txt"
with open(motif_samples_path) as f:
    motif_names = [line.strip() for line in f.readlines()]

# load them as dictionary of PPM matrices
motif_collection = tm.load_motif_collection(motif_collection_dir, motif_names=motif_names)
motif_annotations = tm.load_motif_annotations(motif_annotations_file)

# load motif to dna-binding domain (DBD) mapping
motif_to_db = tm.load_motif_to_dbd(motif_annotations)

## Extracting seqlets using tangermeme

We use `tangermeme` to extract seqlets (spans of nucleotides with high importance scores) from our nucleotide level contribution scores per cell type.  
We use the `tfmindi.pp.extract_seqlets` for this, which wraps `tangermeme`s [recursive_seqlets](https://tangermeme.readthedocs.io/en/latest/tutorials/Tutorial_A4_Seqlets.html#Recursive-Seqlets) functionality.  
The resulting seqlets will be scaled to a range of [-1,1] and sign corrected so the average contribution values are always positive. 

In [ ]:
# extract_seqlets expects the inputs to be in shape (n, region_width, 4), so we concatenate the cell type specific contributions
CONTRIB_FOLDER = DATA_DIR + "modisco_results_ft_2000/"

contrib_list = []
oh_list = []
classes_list = []

class_names = [
    re.match(r"(.+?)_oh\.npz$", f).group(1)  # type: ignore
    for f in os.listdir(CONTRIB_FOLDER)
    if f.endswith("_oh.npz")
]

for i, c in enumerate(tqdm(class_names)):
    contrib_list.append(np.load(os.path.join(CONTRIB_FOLDER, f"{c}_contrib.npz"))["arr_0"])
    oh_list.append(np.load(os.path.join(CONTRIB_FOLDER, f"{c}_oh.npz"))["arr_0"])
    classes_list.append(np.repeat(c, oh_list[i].shape[0]))

oh = np.concatenate(oh_list)
contrib = np.concatenate(contrib_list)
classes = np.concatenate(classes_list)

100%|██████████| 19/19 [01:21<00:00,  4.31s/it]


In [ ]:
# extract seqlets from the contributions and one-hot encoded sequences
seqlets_df, seqlets_matrices = tm.pp.extract_seqlets(contrib=contrib, oh=oh, threshold=0.05, additional_flanks=3)

Processing seqlets: 100%|██████████| 1007193/1007193 [01:30<00:00, 11160.99it/s]


In [ ]:
seqlets_df.head(3)  # information on seqlet position and significance

,example_idx,start,end,attribution,p-value
0,13895,1096,1125,45.111572,4.839551e-11
1,12255,1020,1043,-34.561593,1.637069e-07
2,13577,1067,1097,44.328660,3.006946e-07


In [ ]:
seqlets_matrices[0].shape  # seqlets_matrices is a list with len(seqlets), that contains the scaled matrix per seqlet

(4, 29)

## Calculating motif similarity

Next, we calculate similarity scores between the extracted seqlets and the SCENIC+ motif collection by using `memelite`'s TomTom implementation.  
The resulting similarity matrix will be log-transformed and negated before performing clustering.  

In [ ]:
sim_matrix = tm.pp.calculate_motif_similarity(
    seqlets_matrices, motif_collection, n_nearest=500
)  # use n_nearest if you're running out of memory

MemoryError: Allocation failed (probably too large).